In [1]:

%pprint
import sys
sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

from datetime import timedelta
from frvrs_utils import FRVRSUtilities
from notebook_utils import NotebookUtilities
from pandas import DataFrame
import humanize
import matplotlib.pyplot as plt
import numpy as np
import os
import os.path as osp
import pandas as pd
import re

nu = NotebookUtilities(
    data_folder_path=osp.abspath('../data'),
    saves_folder_path=osp.abspath('../saves')
)
fu = FRVRSUtilities(
    data_folder_path=osp.abspath('../data'),
    saves_folder_path=osp.abspath('../saves')
)

In [ ]:

for sub_directory, directories_list, files_list in os.walk('../../../GitHub'):
    for file_name in files_list:
        if file_name.endswith('.jpg') and ('map' in file_name.lower()):
            file_path = osp.join(sub_directory, file_name)
            print(file_path)

In [ ]:

for root_dir in ['C:\\', 'D:\\']:
    for sub_directory, directories_list, files_list in os.walk(root_dir):
        if 'ffmpeg' in sub_directory.split(os.sep)[-1].lower():
            print(sub_directory)
            !start %windir%\explorer.exe "{sub_directory}"

In [ ]:

print([fn for fn in dir(os) if 'dir' in fn.lower()])
[fn.split('.')[0] for fn in os.listdir(nu.saves_pickle_folder)]

In [ ]:

columns_set = set()
for pickle_name in [fn.split('.')[0] for fn in os.listdir(s.saves_pickle_folder)]:
    if pickle_name.endswith('_df'):
        df = nu.load_object(pickle_name)
        columns_set.update(df.columns[list(map(lambda x: ('year' in x) and (('begin' in x) or ('end' in x)), df.columns))])
columns_set

In [7]:

# 'Intel64 Family 6 Model 60 Stepping 3, GenuineIntel'
import platform

platform.processor()

'AMD64 Family 25 Model 80 Stepping 0, AuthenticAMD'

In [9]:

import re

req_str = 'Home Insurance  BINDER'
'_'.join(re.split(r'[ \\\/:\*\?"><\|]+', req_str, 0))

'Home_Insurance_BINDER'

In [ ]:

for root_dir in ['C:\\', 'D:\\']:
    for sub_directory, directories_list, files_list in os.walk(root_dir):
        for file_name in files_list:
            if 'ffmpeg' in file_name.lower():
                file_path = osp.join(sub_directory, file_name)
                print(file_path)

In [ ]:

for root_dir in ['C:\\', 'D:\\']:
    for sub_directory, directories_list, files_list in os.walk(root_dir):
        if 'kml' in sub_directory.split(os.sep)[-1].lower():
            print(sub_directory)
            !start %windir%\explorer.exe "{sub_directory}"
        else:
            for file_name in files_list:
                if 'kml' in file_name.lower():
                    file_path = osp.join(sub_directory, file_name)
                    print(file_path)

In [ ]:

magic_dict_list = magic_dict['test.py']
print(len(magic_dict_list))
subprocess.run([comparator_path, osp.abspath(magic_dict_list[0]), osp.abspath(magic_dict_list[1])])

In [10]:

import sys

for key, value in sys.modules.items():
    if 'xdist' in key.lower():
        #print('{}: {}'.format(key, value))
        print('{}'.format(key))

In [ ]:

for root_dir in ['C:\\', 'D:\\']:
    for sub_directory, directories_list, files_list in os.walk(root_dir):
        for file_name in files_list:
            if file_name == 'git.exe':
                file_path = osp.join(sub_directory, file_name)
                print('Write-Host "#################################################################################"')
                print('Write-Host "    Configuring {}"'.format(file_path))
                print('Write-Host "#################################################################################"')
                print('cd "{}"'.format(sub_directory))
                print('git.exe config --global core.autocrlf input')